<a href="https://colab.research.google.com/github/yukinaga/image_generation/blob/main/section_5/01_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSGANによる画像素材の生成
LSGANにより、128x128の画像素材を生成します。

## データの読み込み
多数の犬猫画像が格納された、Pet Datasetをダウンロードし、解凍します。  
https://www.robots.ox.ac.uk/~vgg/data/pets/  


In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz -P ./data  # ダウンロード
!tar -zxvf ./data/images.tar.gz -C ./data  # 解凍

## 各設定
LSGANに必要な各設定を行います。 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

img_size = 128  # 入力画像の高さと幅
n_noise = 64  # ノイズの数

epochs = 1200 # 学習回数
interval = 20  # 経過の表示間隔
batch_size = 128

transform = transforms.Compose([
    # transforms.Resize((img_size, img_size)),  # サイズの変更
    transforms.CenterCrop(256),  # 画像の中央を取り出し
    transforms.RandomCrop(128),  # ランダムに切り抜き
    transforms.RandomHorizontalFlip(),  # ランダムに左右反転
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # -1から1の範囲に
])

# データセットの設定
pets_datasets = datasets.ImageFolder(
    root= "./data", 
    transform=transform
    )
print("画像の枚数:", len(pets_datasets))

# DataLoaderの設定
train_loader = DataLoader(pets_datasets, 
                          batch_size=batch_size,
                          shuffle=True,
                           num_workers=2
                          )

## 画像データの確認
画像データを並べて表示し、確認します。

In [ ]:
image = iter(train_loader).next()[0]
image = image.cpu().detach().numpy().transpose(0, 2, 3, 1)  # (バッチ、行、列、チャンネル)に変更
image = image/2+0.5  # 0-1の範囲に

rows = 3  # 行数
columns = 5  # 列数
scale = 4  # 表示スケール
plt.figure(figsize=(scale*columns, scale*rows))

for i in range(rows*columns):
    ax = plt.subplot(rows, columns, i+1)
    plt.imshow(image[i])
plt.show()

## Generatorの構築
PyTorchによりGeneratorのモデルを構築します。  
Generatorでは畳み込みの逆を行い、ノイズから画像を生成します。  
今回は畳み込みの逆を行う層を6層重ねます。また、途中でデータが偏らないようにBatch Normalizationを導入します。  
これらは、`ModuleList`により学習可能な層としてリストに格納します。  



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layers = nn.ModuleList([
            # 画像サイズ 1x1→4x4
            nn.ConvTranspose2d(n_noise, 512, 4, 1, 0),  # 入力のチャンネル数, 出力のチャンネル数, カーネルのサイズ、ストライド、パディング
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 画像サイズ 4x4→8x8
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # 画像サイズ 8x8→16x16
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # 画像サイズ 16x16→32x32
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
           # 画像サイズ 32x32→64x64
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # 画像サイズ 64x64→128x128
            nn.ConvTranspose2d(32, 3, 4, 2, 1),
            nn.Tanh(),
        ])

    def forward(self, x):
        x = x.view(-1, n_noise, 1, 1)  # (バッチサイズ, チャンネル数, 高さ, 幅)
        for layer in self.layers:
            x = layer(x)
        return x

generator = Generator()
generator.cuda()  # GPU対応
print(generator)

## Discriminatorの構築
Discriminatorのモデルを構築します。   
今回は畳み込みを行う層を6層重ねます。また、途中でデータが偏らないようにBatch Normalizationを導入します。  
LSGANでは、最後の層の活性化関数にはsigmoid関数ではなく恒等関数を使います。 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.ModuleList([
            # 画像サイズ 128x128→64x64
            nn.Conv2d(3, 32, 4, 2, 1),  # 入力のチャンネル数, 出力のチャンネル数, カーネルのサイズ
            nn.LeakyReLU(negative_slope=0.2),
            # 画像サイズ 64x64→32x32
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2),
            # 画像サイズ 32x32→16x16
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.2),
            # 画像サイズ 16x16→8x8
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2),
           # 画像サイズ 8x8→4x4
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.2),
            # 画像サイズ 4x4→1x1
            nn.Conv2d(512, 1, 4, 1, 0),
        ])

    def forward(self, x):
        x = x.view(-1, 3, img_size, img_size)  # (バッチサイズ, チャンネル数, 高さ, 幅)
        for layer in self.layers:
            x = layer(x)
        x = x.view(-1, 1)  # (バッチサイズ, 出力の数)
        return x

discriminator = Discriminator()
discriminator.cuda()  # GPU対応
print(discriminator)

### 画像の生成
画像を生成して表示するための関数を定義します。  
画像は、訓練済みのGenertorにノイズを入力することで生成されます。  
画像は4×4枚生成されますが、並べて一枚の画像にした上で表示されます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -- 画像を生成して表示 --
def generate_images(i):
    # 画像の生成
    n_rows = 4  # 行数
    n_cols = 4  # 列数
    noise = torch.randn(n_rows * n_cols, n_noise).cuda()
    g_imgs = generator(noise)
    g_imgs = g_imgs/2 + 0.5  # 0-1の範囲にする
    g_imgs = g_imgs.cpu().detach().numpy()

    img_size_spaced = img_size + 2
    matrix_image = np.zeros((img_size_spaced*n_rows, img_size_spaced*n_cols, 3))  # 全体の画像

    #  生成された画像を並べて一枚の画像にする
    for r in range(n_rows):
        for c in range(n_cols):
            g_img = g_imgs[r*n_cols + c].transpose(1, 2, 0).reshape(img_size, img_size, 3)
            top = r*img_size_spaced
            left = c*img_size_spaced
            matrix_image[top : top+img_size, left : left+img_size, :] = g_img

    plt.figure(figsize=(8, 8))
    plt.imshow(matrix_image, vmin=0.0, vmax=1.0)
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # 軸目盛りのラベルと線を消す
    plt.show()

## 正解数の計算
Discriminatorによる鑑定の正解数を、カウントする関数を定義します。  
Discriminatorの精度の計算に使用します。

In [ ]:
def count_correct(y, t):
    correct = torch.sum((torch.where(y<0.5, 0, 1) ==  t).float())
    return correct.item()

## 学習
構築したLSGANのモデルを使って、学習を行います。  
Generatorが生成した画像には正解ラベル0、本物の画像には正解ラベル1を与えてDiscriminatorを訓練します。  
その後にGeneratorを訓練しますが、この場合の正解ラベルは1になります。  
LSGANでは、誤差に二乗誤差を使用します。  

In [ ]:
from torch import optim

# 平均二乗誤差
loss_func = nn.MSELoss()

# Adam
optimizer_gen = optim.Adam(generator.parameters())
optimizer_disc = optim.Adam(discriminator.parameters())

# ログ
error_record_fake = []  # 偽物画像の誤差記録
acc_record_fake = []  # 偽物画像の精度記録
error_record_real = []  # 本物画像の誤差記録
acc_record_real = []  # 本物画像の精度記録

# -- DCGANの学習 --
generator.train()
discriminator.train()
for i in range(epochs):
    loss_fake = 0  # 誤差
    correct_fake = 0  # 正解数
    loss_real = 0
    correct_real = 0
    n_total = 0  # データの総数（精度の計算に使用）
    for j, (x, t) in enumerate(train_loader):  # ミニバッチ（x,）を取り出す

        n_total += x.size()[0]  # バッチサイズを累積

        # ノイズから画像を生成しDiscriminatorを訓練
        noise = torch.randn(x.size()[0], n_noise).cuda()
        imgs_fake = generator(noise)  # 画像の生成
        t = torch.zeros(x.size()[0], 1).cuda()  # 正解は0
        y = discriminator(imgs_fake)
        loss = loss_func(y, t)
        optimizer_disc.zero_grad()
        loss.backward()
        optimizer_disc.step()  # Discriminatorのみパラメータを更新
        loss_fake += loss.item()
        correct_fake += count_correct(y, t)

        # 本物の画像を使ってDiscriminatorを訓練
        imgs_real= x.cuda()
        t = torch.ones(x.size()[0], 1).cuda()  # 正解は1
        y = discriminator(imgs_real)
        loss = loss_func(y, t)
        optimizer_disc.zero_grad()
        loss.backward()
        optimizer_disc.step()  # Discriminatorのみパラメータを更新
        loss_real += loss.item()
        correct_real += count_correct(y, t)

        # Generatorを訓練
        # noise = torch.randn(x.size()[0]*2, n_noise).cuda()  # バッチサイズを2倍にする
        imgs_fake = generator(noise)  # 画像の生成
        t = torch.ones(x.size()[0]*1, 1).cuda()  # 正解は1
        y = discriminator(imgs_fake)
        loss = loss_func(y, t)
        optimizer_gen.zero_grad()
        loss.backward()
        optimizer_gen.step()  # Generatorのみパラメータを更新

    loss_fake /= j+1  # 誤差
    error_record_fake.append(loss_fake)
    acc_fake = correct_fake / n_total  # 精度
    acc_record_fake.append(acc_fake)

    loss_real /= j+1  # 誤差
    error_record_real.append(loss_real)
    acc_real = correct_real / n_total  # 精度
    acc_record_real.append(acc_real)

    # 一定間隔で誤差と精度、および生成された画像を表示
    if i % interval == 0:
        print ("Epochs:", i)
        print ("Error_fake:", loss_fake , "Acc_fake:", acc_fake)
        print ("Error_real:", loss_real , "Acc_real:", acc_real)
        generate_images(i)

### 誤差と正解率の推移
学習中における、誤差と正解率の推移を確認します。  
Discriminatorに本物画像を鑑定させた際の誤差の推移と、偽物画像を鑑定させた際の誤差の推移をグラフに表示します。  
正解率の推移も表示します。  

In [ ]:
# -- 誤差の推移 --
plt.plot(range(len(error_record_fake)), error_record_fake, label="Error_fake")
plt.plot(range(len(error_record_real)), error_record_real, label="Error_real")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

# -- 正解率の推移 --
plt.plot(range(len(acc_record_fake)), acc_record_fake, label="Acc_fake")
plt.plot(range(len(acc_record_real)), acc_record_real, label="Acc_real")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

## 演習
他のデータセットを使って学習を行い、本物らしい画像が生成されることを確認しましょう。  

